In [55]:
import json
import geopy.distance
import pandas as pd
import geopandas as gpd
import numpy as np

print("Pandas Version: " + pd.__version__)
print("Geopandas Version: " + gpd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.0.1
Geopandas Version: 0.6.2
Numpy Version: 1.18.2


## StadtRAD Buchungen

Der Datensatz `OPENDATA_BOOKING_CALL_A_BIKE.csv` enthält Informationen über Buchungen, die bei call-a-bike in der Zeit zwischen dem 01.01.2014 und dem 16.05.2017 Uhr notiert wurden. Im Folgenden wird dieser Datensatz auf Fehler untersucht und für die Visualisierung auf der Mobility Map vorbereitet.

In [56]:
df = pd.read_csv("../data/call_a_bike/OPENDATA_BOOKING_CALL_A_BIKE.csv", sep=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16228298 entries, 0 to 16228297
Data columns (total 17 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   BOOKING_HAL_ID             int64  
 1   CATEGORY_HAL_ID            int64  
 2   VEHICLE_HAL_ID             int64  
 3   CUSTOMER_HAL_ID            object 
 4   DATE_BOOKING               object 
 5   DATE_FROM                  object 
 6   DATE_UNTIL                 object 
 7   COMPUTE_EXTRA_BOOKING_FEE  object 
 8   TRAVERSE_USE               object 
 9   DISTANCE                   float64
 10  START_RENTAL_ZONE          object 
 11  START_RENTAL_ZONE_HAL_ID   float64
 12  END_RENTAL_ZONE            object 
 13  END_RENTAL_ZONE_HAL_ID     float64
 14  RENTAL_ZONE_HAL_SRC        object 
 15  CITY_RENTAL_ZONE           object 
 16  TECHNICAL_INCOME_CHANNEL   object 
dtypes: float64(3), int64(3), object(11)
memory usage: 2.1+ GB


In [57]:
df.sample(10)

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL
2382483,24870373,53000,116725,843C9B6184BE01253CB49BE1C318984EFBC60F43,2014-07-26 22:13:50,2014-07-26 22:13:50,2014-07-26 22:21:09,Nein,Nein,0.0,Weserspitze / Weserstraße,204816.0,Weserspitze / Weserstraße,204816.0,Standort,Kassel,iPhone KON
13447195,41651748,51000,118991,24E4595A651F42BF9A14B0528A47B60A10FBC59C,2016-09-25 19:04:35,2016-09-25 19:04:35,2016-09-25 19:13:06,Nein,Nein,0.0,Saarlandstraße/Wiesendamm,203890.0,Weidestraße/Biedermannplatz,211922.0,Standort,Hamburg,Android SRH
11596902,39121280,50000,164471,E04E061986C3C89DDF0F6718600FD44884A5D4C9,2016-07-04 16:28:05,2016-07-04 16:28:05,2016-07-04 16:48:03,Nein,Nein,0.0,Europaallee / Den Haager Straße,47276.0,Kelsterbacher Straße / Schwanheimer Straße,254657.0,Standort,Frankfurt am Main,Android CAB
11454196,38923634,50000,116549,A5D55764D75C3B91F3582CD6998598C233ED887D,2016-06-27 19:49:48,2016-06-27 19:49:48,2016-06-27 19:58:37,Nein,Nein,0.0,NaN,6318.0,NaN,NaN,Standort,Köln,Android CAB
5443572,29716728,51000,117574,E9ABBD02D2E18B4D5D840E90B151DF4324D0E9D0,2015-05-12 23:17:36,2015-05-12 23:17:36,2015-05-12 23:27:54,Nein,Nein,0.0,Hudtwalckerstraße / Bebelallee,138378.0,Hofweg/Am Langenzug,200502.0,Standort,Hamburg,Android SRH
5262344,29445429,53000,117110,32FB3FB0866152F6B6ECD41D13FDCC2CD1D53C5D,2015-05-01 17:36:35,2015-05-01 17:36:35,2015-05-01 17:56:10,Nein,Nein,0.0,Königsplatz Ost,204807.0,Uni-Kassel / Menzelstr.,204785.0,Standort,Kassel,iPhone KON
12670088,40596963,50000,165808,D83D0EC85833E0A18068F69A7AD26AC885B49AAD,2016-08-22 21:28:59,2016-08-22 21:28:59,2016-08-22 21:43:00,Nein,Nein,0.0,NaN,34992.0,NaN,NaN,Standort,Frankfurt am Main,iPhone CAB
9061618,35242376,51000,120184,A71F771C3A5D81079EBC1874B9F0C3EC4A45BE95,2016-01-23 14:39:01,2016-01-23 14:39:01,2016-01-23 14:56:40,Nein,Nein,0.0,Hudtwalckerstraße / Bebelallee,138378.0,Bahnhof Dammtor Nord // Theodor-Heuss-Platz,131883.0,Standort,Hamburg,Android SRH
12402191,40224868,51000,115729,A126790CBCFCD7398E1476B0D7975EF278AB904B,2016-08-09 18:59:53,2016-08-09 18:59:53,2016-08-09 19:00:32,Nein,Nein,0.0,Osterstraße/Bismarckstraße,131642.0,Osterstraße/Bismarckstraße,131642.0,Standort,Hamburg,iPhone CAB
7620087,32833648,51000,117224,B91BB506796C872F1C75D6EE240CB66B9C5EC964,2015-09-10 21:11:22,2015-09-10 21:11:22,2015-09-10 21:19:05,Nein,Nein,0.0,Burgstraße/Hammer Landstraße,131639.0,Hasselbrookstraße/Papenstraße,204031.0,Standort,Hamburg,Android SRH


Der Wert `CITY_RENTAL_ZONE` gibt an, in welcher Stadt ein Eintrag gemacht wurde. Dieses Projekt konzentriert sich auf die Mobilität in Hamburg, daher werden Daten aus anderen Städten entfernt.

In [58]:
bookings_raw = df[df.CITY_RENTAL_ZONE == "Hamburg"].dropna()

Bei einer oberflächigen Sichtung der Daten ist bereits bei einigen Spalten aufgefallen, dass ihr Nutzen zweifelhaft ist. Diese werden nun auf Fehler oder überflüssige Inhalte untersucht. Es wird Folgendes ausgegeben:

1. Anzahl der Buchungen, bei denen das Datum `DATE_BOOKING` nicht gleich dem Datum `DATE_FROM` ist.
2. Anzahl der Buchungen, bei denen der Wert `DATE_FROM` gleich dem Wert `DATE_UNTIL` ist.
3. Anzahl der Buchungen, bei denen der Wert `DISTANCE` größer 0 ist.
4. Alle vorkommenden Werte in der Spalte `TRAVERSE_USE`.

In [59]:
x1 = (bookings_raw[bookings_raw['DATE_BOOKING'] != bookings_raw['DATE_FROM']]).shape[0]
print("1. DATE_BOOKING != DATE_FROM: " + str(x1))

x2 = (bookings_raw[bookings_raw['DATE_FROM'] >= bookings_raw['DATE_UNTIL']]).shape[0]
print("2. DATE_FROM >= DATE_UNTIL: " + str(x2))

x3 = (bookings_raw[bookings_raw['DISTANCE'] != 0.0]).shape[0]
print("3. DISTANCE != 0: " + str(x3))

print("4. TRAVERSE_USE:  {}".format(bookings_raw.TRAVERSE_USE.unique()))

1. DATE_BOOKING != DATE_FROM: 0
2. DATE_FROM >= DATE_UNTIL: 0
3. DISTANCE != 0: 0
4. TRAVERSE_USE:  ['Nein']


Daraus können die folgenden Erkenntnisse gewonnen werden:

1. Der Zeitpunkt, an dem die Buchung begonnen wurde (`DATE_BOOKING`), ist in jedem Fall gleich dem Zeitpunkt der Ausleihe (`DATE_FROM`). Daher kann die Spalte `DATE_BOOKING` entfernt werden.
2. Es gibt keine Einträge, bei denen der Zeitpunkt der Ausleihe (`DATE_FROM`) nach dem Zeitpunkt der Rückgabe liegt (`DATE_UNTIL`). In dieser Hinsicht weist der Datensatz also keine Inkonsistenzen auf. Weiterhin lässt sich erkennen, dass jede Buchung eine Dauer von mindestens einer Sekunde hat.
3. Die Spalte `DISTANCE` besitzt in keinem Fall einen Wert größer 0 und ist somit unnütz.
4. Die Spalte `TRAVERSE_USE` TRAVERSE_USE in jedem Fall den Wert `Nein` und ist somit ebenfalls unnütz.

Im Folgenden werden unnötige Spalten entfernt. Die verbleibenden Spalten bekommen neue Bezeichnungen, die etwas lesbarer sind.

In [60]:
bookings = bookings_raw[[
    "VEHICLE_HAL_ID",
    "CUSTOMER_HAL_ID",
    "DATE_FROM",
    "DATE_UNTIL",
    "START_RENTAL_ZONE",
    "START_RENTAL_ZONE_HAL_ID",
    "END_RENTAL_ZONE",
    "END_RENTAL_ZONE_HAL_ID",
    "TECHNICAL_INCOME_CHANNEL",
]].copy()
bookings.columns = [
    "bike_id",
    "user_id",
    "date_from",
    "date_until",
    "from_station_name", 
    "from_station_id",
    "to_station_name",
    "to_station_id",
    "booked_via"
]

Angelehnt an das Abrechnungsmodell von StadtRAD wird nun für jeden Eintrag die Dauer der Buchung in angefangenen Minuten berechnet. Die Dauer wird in einer neuen Spalte `duration_in_min` gespeichert.

In [61]:
def calculate_duration_in_min(date_from, date_until):
    duration_in_seconds = pd.to_datetime(date_until) - pd.to_datetime(date_from)
    return np.ceil(duration_in_seconds / pd.Timedelta(minutes=1))

bookings.loc[:, "duration_in_min"] = calculate_duration_in_min(bookings["date_from"], bookings["date_until"])

bookings.sample(3)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min
10622637,109222,B3E24635269B26EDE634FB1C879F8823EB65813E,2016-05-20 15:40:24,2016-05-20 16:36:45,Gänsemarkt / Büschstraße,131884.0,Gänsemarkt / Büschstraße,131884.0,IVR,57.0
7048076,110645,32D9F8E9572CB013E708E4F057125CCB66D7E851,2015-08-09 14:04:17,2015-08-09 14:23:11,Uferstraße/Wagnerstraße,207619.0,Eduard-Rhein-Ufer / Schwanenwik,140800.0,Terminal HH_112 (-2331-),19.0
14245241,111087,9F747E24912DEF11081CFC1C42B267A6C7D4373F,2016-11-22 09:49:53,2016-11-22 09:56:52,Neunenfelder Straße/ Kurt-Emmerich-Platz,226607.0,Georg-Wilhelm-Straße/Mankiewicz,251476.0,Android SRH,7.0


Bei den Werten der Spalten `from_station_id`, `to_station_id` und `duration_in_min` handelt es sich momentan noch um Gleitkommazahlen. Hier würden ganze Zahlen jedoch mehr Sinn machen, daher werden die Werte konvertiert.

In [62]:
bookings["from_station_id"] = bookings["from_station_id"].astype(int)
bookings["to_station_id"] = bookings["to_station_id"].astype(int)
bookings["duration_in_min"] = bookings["duration_in_min"].astype(int)

Jede Buchung soll später auf der Mobility Map als eine Verbindung zwischen zwei Punkten dargestellt werden (Start und Ziel). Um dies mit "kepler.gl" zu vereinfachen, werden zu jedem Eintrag die Geo-Koordinaten der Ausleih- und Rückgabestation hinzugefügt. Jede Zeile enthält somit den Längen- und Breitengrad beider Stationen in den Spalten `from_station_lat`, `from_station_lng`, `to_station_lat` und `to_station_lng`. Die Koordinaten werden dem Datensatz zu den StadtRAD Stationen übernommen, welcher vorher in einem weiteren Notebook aufbereitet wurde.

In [63]:
stations = gpd.read_file("../data/call_a_bike/stations.geojson")
latitudes = {}
longitudes = {}
for index, row in stations.iterrows():
    latitudes[row["station_id"]] = row["latitude"]
    longitudes[row["station_id"]] = row["longitude"]

In [64]:
bookings["from_station_lat"] = bookings["from_station_id"].map(latitudes)
bookings["from_station_lng"] = bookings["from_station_id"].map(longitudes)
bookings["to_station_lat"] = bookings["to_station_id"].map(latitudes)
bookings["to_station_lng"] = bookings["to_station_id"].map(longitudes)
bookings["date_from"] = pd.to_datetime(bookings["date_from"])
bookings["date_until"] = pd.to_datetime(bookings["date_until"])
bookings.sample(3)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,from_station_lat,from_station_lng,to_station_lat,to_station_lng
9294372,108685,E0F121AD3486D39104A27959E317E3CEA2CE4499,2016-02-17 17:01:28,2016-02-17 17:09:08,Steindamm / Stralsunder Straße,138389,Alsterschwimmhalle/Ifflandstraße,198081,Android SRH,8,53.554216,10.013306,53.560781,10.022148
13238516,119931,EF816002FF2E101CAB625CDEBC29556780D37F34,2016-09-15 19:01:29,2016-09-15 19:24:01,Bernhard-Nocht-Straße/Zirkusweg,196417,Wartenau / Lübecker Straße,140801,Android SRH,23,53.547320,9.965669,53.563600,10.034100
11229985,107774,62DF6CBFD2BCBB953CBE2B8144AC6B5BF7BAEB91,2016-06-17 08:56:37,2016-06-17 09:14:11,Michaeliskirche / Krayenkamp,138386,Eimsbütteler Straße/Waterloostraße,131644,IVR,18,53.547975,9.979420,53.566200,9.953400


Nun wird geprüft, ob für jede Station Koordinaten ermittelt werden konnten. Durch das Zählen der leeren (NaN) Einträge wird klar, das dies nicht der Fall ist.

In [65]:
bookings.isna().sum()

bike_id                  0
user_id                  0
date_from                0
date_until               0
from_station_name        0
from_station_id          0
to_station_name          0
to_station_id            0
booked_via               0
duration_in_min          0
from_station_lat     41031
from_station_lng     41031
to_station_lat       49951
to_station_lng       49951
dtype: int64

Für die weitere Verarbeitung sollen Zeilen mit leeren Werten entweder vervollständigt oder entfernt werden. Da noch ein weiterer Datensatz [des European Dataportal](https://www.europeandataportal.eu/data/datasets/d18f375e-fa5f-4998-aff8-557969f44479) zu StadtRAD Stationen vorhanden ist (abgelegt unter `data/europeandataportal/Stadtrad`) können fehlende Koordinaten eventuell ergänzt werden. Da in diesem Datensatz nur die Namen und nicht die Nummern der Stationen enthalten ist, werden die Namen der Stationen mit fehlenden Daten ermittelt.

In [66]:
missing_from_stations = bookings[bookings.from_station_lng.isnull()]
missing_to_stations = bookings[bookings.to_station_lng.isnull()]
tmp = sorted(missing_from_stations.from_station_name.unique()) + sorted(missing_to_stations.to_station_name.unique())
sorted(set(tmp))

['BSC Testterminal Halle',
 'BSCTerminal FFM 2.OG R 2.21',
 'F PS 1763 / BSC Hamburg Mobil',
 'F PS 2423 / BSC Hamburg Mobil',
 'F R  5156 / BSC Hamburg Mobil',
 'Funkbox-BSC-HH',
 'Grindelberg / Bezirksamt Eimsbüttel',
 'Maretstraße/Baererstraße',
 'Rotec Schlösser BSC',
 'Systemtechnik/Test Terminal']

Über die Gründe dafür, dass diese Stationen nicht im ursprünglichen Datensatz der Deutschen Bahn enthalten sind, lässt sich nur mutmaßen. Es scheint jedoch so als wären in den Daten auch Buchungen enthalten, die zu Testzwecken gemacht wurden, zum Beispiel über die Station "Systemtechnik/Test Terminal". Die beiden Stationen "Grindelberg / Bezirksamt Eimsbüttel" und "Maretstraße/Baererstraße" sind auch im Datensatz des European Dataportal enthalten, sodass sich für diese Einträge Koordinaten ergänzen lassen. Vermutlich handelt es sich hierbei um neuere Stationen, die im Datensatz der Deutschen Bahn noch nicht ergänzt wurden.

In [67]:
latitude_grindelberg = 53.575408
longitude_grindelberg = 9.977824
latitude_maretstrasse = 53.453704
longitude_maretstrasse = 9.982275

bookings.loc[bookings.from_station_name == "Grindelberg / Bezirksamt Eimsbüttel", "from_station_lat"] = latitude_grindelberg
bookings.loc[bookings.from_station_name == "Grindelberg / Bezirksamt Eimsbüttel", "from_station_lng"] = longitude_grindelberg
bookings.loc[bookings.to_station_name == "Grindelberg / Bezirksamt Eimsbüttel", "to_station_lat"] = latitude_grindelberg
bookings.loc[bookings.to_station_name == "Grindelberg / Bezirksamt Eimsbüttel", "to_station_lng"] = longitude_grindelberg

bookings.loc[bookings.from_station_name == "Maretstraße/Baererstraße", "from_station_lat"] = latitude_maretstrasse
bookings.loc[bookings.from_station_name == "Maretstraße/Baererstraße", "from_station_lng"] = longitude_maretstrasse
bookings.loc[bookings.to_station_name == "Maretstraße/Baererstraße", "to_station_lat"] = latitude_maretstrasse
bookings.loc[bookings.to_station_name == "Maretstraße/Baererstraße", "to_station_lng"] = longitude_maretstrasse

bookings.isna().sum()

bike_id                  0
user_id                  0
date_from                0
date_until               0
from_station_name        0
from_station_id          0
to_station_name          0
to_station_id            0
booked_via               0
duration_in_min          0
from_station_lat      3970
from_station_lng      3970
to_station_lat       13846
to_station_lng       13846
dtype: int64

Die restlichen Zeilen mit fehlenden Einträgen werden anschließend aus dem Datensatz entfernt und gesondert gespeichert. So können diese ggf. zu einem späteren Zeitpunkt noch einmal überprüft werden.

In [68]:
unknown_stations = bookings[bookings.isnull().any(axis=1)]
bookings.dropna(inplace=True)
bookings.isna().sum()

bike_id              0
user_id              0
date_from            0
date_until           0
from_station_name    0
from_station_id      0
to_station_name      0
to_station_id        0
booked_via           0
duration_in_min      0
from_station_lat     0
from_station_lng     0
to_station_lat       0
to_station_lng       0
dtype: int64

Da für jede Buchung jetzt die Start- und Zielkoordinaten bekannt sind, kann die lineare Distanz zwischen den beiden Punkten berechnet werden. Dies stellt nicht dar, wieviel tatsächlich geradelt wurde. Die Entfernung zwischen Start- und Zielstation kann jedoch trotzdem interessant sein.

In [69]:
# Wird weggelassen, da zu rechenintensiv
#def calculate_linear_distance_in_m(row):
#    coords_a = (row["from_station_lat"], row["from_station_lng"])
#    coords_b = (row["to_station_lat"], row["to_station_lng"])
#    distance = geopy.distance.distance(coords_a, coords_b)
#    return round(distance.m)
#
#bookings["linear_distance_in_m"] = bookings.apply(calculate_linear_distance_in_m, axis=1)
#bookings.sample(3)

Nun wird nun die Spalte `booked_via` genauer untersucht. Die entsprechenden Werte geben vermutlich an, ob ein StadtRAD über das Mobiltelefon, über das Terminal an der Station oder über weitere Channel gebucht wurde. Um die Werte genauer betrachten zu können wird eine sortierte Liste aller einmalig vorkommenden Werte ausgegeben.

In [70]:
sorted(bookings.booked_via.unique())

['Android',
 'Android CAB',
 'Android KON',
 'Android SRH',
 'BlackBerry',
 'CaB - Mobility Map',
 'IVR',
 'LIDL-BIKE',
 'Starenkasten HH_193 (-275-)',
 'Starenkasten HH_211 (-289-)',
 'Starenkasten HH_236 (-2128-)',
 'Starenkasten HH_24 (-272-)',
 'Techniker HH_115 (-2124-)',
 'Techniker HH_116 (-2125-)',
 'Techniker HH_117 (-2813-)',
 'Techniker HH_118 (-2332-)',
 'Techniker HH_119 (-2334-)',
 'Techniker HH_12 (-2333-)',
 'Techniker HH_121 (-2551-)',
 'Techniker HH_122 (-2681-)',
 'Techniker HH_123 (-2562-)',
 'Techniker HH_124 (-2383-)',
 'Techniker HH_125 (-2224-)',
 'Techniker HH_126 (-2251-)',
 'Techniker HH_127 (-2623-)',
 'Techniker HH_128 (-2814-)',
 'Techniker HH_129 (-2381-)',
 'Techniker HH_131 (-2216-)',
 'Techniker HH_132 (-2512-)',
 'Techniker HH_133 (-2382-)',
 'Techniker HH_134 (-2413-)',
 'Techniker HH_135 (-2151-)',
 'Techniker HH_136 (-2221-)',
 'Techniker HH_138 (-2244-)',
 'Techniker HH_139 (-2351-)',
 'Techniker HH_14 (-2337-)',
 'Techniker HH_141 (-2712-)',
 'Te

Dabei fällt auf, dass viele Einträge mit dem Suffix "Techniker" beginnen. Anzahl:

In [71]:
bookings[bookings.booked_via.str.startswith("Techniker", True)].shape[0]

728094

Hierbei handelt es sich vermutlich um Buchungen, die von call-a-bike Technikern zwecks Instandhaltung der Räder gemacht wurden. Sie spiegeln nicht das Ausleihverhalten der Kunden wider, könnten aber trotzdem interessant sein. Der Datensatz wird daher nach den entsprechenden Werten der Spalte `booked_via` in zwei Datensätze aufgeteilt.

In [72]:
maintenance = bookings[bookings["booked_via"].str.contains("Techniker")]
bookings = pd.concat([bookings, maintenance]).drop_duplicates(keep=False)

Der Datensatz `bookings` enthält jetzt nur noch Kundenfahrten, der Datensatz `maintenance` enthält die Technikerdaten. Zusätzlich enthält der Datensatz `unknown_stations` Daten, bei denen die Position der Stationen nicht bekannt ist. Um die jeweiligen Lücken in den Indexen zu schließen werden diese zurückgesetzt.

In [73]:
bookings.reset_index(drop=True, inplace=True)
bookings.sort_index(inplace=True)

bookings.sample(5)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,from_station_lat,from_station_lng,to_station_lat,to_station_lng
1549168,119953,E0DED3C64D4AE83E17D3EC26CBF6BFDEF304C4E4,2014-08-30 00:33:24,2014-08-30 01:01:24,Paulinenplatz/Wohlwillstraße,131547,Bei der Hammer Kirche/Hammerpark,212078,IVR,28,53.554200,9.962460,53.556699,10.055075
6850296,120282,208F57B7A4F400F29E6FE600B40B4F27A992AA16,2016-10-22 11:35:20,2016-10-22 12:13:44,Koppelstraße/Lokstedter Grenzstraße,213792,Kellinghusenstraße / Loogeplatz,140795,iPhone SRH,39,53.593587,9.943525,53.588516,9.991125
5637609,120346,0D900655B48A422070A83DF8AC75A588E36D12DC,2016-06-19 15:18:00,2016-06-19 15:59:49,Siemersplatz/Kollaustraße,252484,Messehallen / Marktstraße,131888,iPhone SRH,42,53.600072,9.963929,53.558518,9.975346
5866022,116037,A4685FCFF71A4D786AC385E09264FF1952DA02AF,2016-07-13 17:44:25,2016-07-13 17:54:06,Mönckebergstraße / Steintorwall,138387,Jungfernstieg / Ballindamm,131879,iPhone SRH,10,53.551600,10.005400,53.552244,9.995501
2335200,143712,90985E52863FA72162D153CB864961D9EA2BB521,2015-02-05 10:01:35,2015-02-05 10:10:15,Jarrestraße / Rambatzweg,138376,Planetarium/Hindenburgstraße,213688,Terminal HH_75 (-2364-),9,53.584000,10.021100,53.595950,10.013650


In [74]:
maintenance.reset_index(drop=True, inplace=True)
maintenance.sort_index(inplace=True)

maintenance.sample(5)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,from_station_lat,from_station_lng,to_station_lat,to_station_lng
367825,120529,02BF6B29C323DA483B03071445913AE0AE8EF708,2016-01-01 15:04:39,2016-01-01 15:13:26,Koppelstraße/Lokstedter Grenzstraße,213792,Methfesselstraße/Luruper Weg,201324,Techniker HH_126 (-2251-),9,53.593587,9.943525,53.580060,9.946786
643728,107679,9596A23FC10C21889DF53DC5A120806B69C2D1C6,2016-12-13 12:19:15,2016-12-13 12:26:16,Amsinckstraße/ Nordkanalbrücke,218174,Hauptbahnhof West / Glockengießer Wall,131872,Techniker HH_159 (-2642-),8,53.547868,10.012361,53.553676,10.004553
301372,115988,35028C82DC658B3DFA4A58348D166D2365204BBB,2015-09-02 18:10:59,2015-09-02 18:33:04,Emilienstraße/Fruchtallee,213825,Am Kaiserkai/Großer Grasbrook,193238,Techniker HH_125 (-2224-),23,53.571690,9.953138,53.541838,9.992856
96885,120816,31E9B2D07775575A5D84668839BAB9F2532AA408,2014-07-28 22:46:03,2014-07-28 22:51:32,Wandsbeker Chaussee/ Börnestraße,227415,Schloßstrasse/Schloßgarten,207304,Techniker HH_134 (-2413-),6,53.570470,10.059528,53.571664,10.068644
206075,119835,F586FD4823756C59FDF8951E8FD07CAAB402375C,2015-05-06 19:15:55,2015-05-06 19:28:23,Gotthelfweg/Außenmühlenteich,242732,Moorstraße/Bahnhof Harburg,242358,Techniker HH_15 (-2841-),13,53.447625,9.980675,53.457286,9.987994


In [75]:
unknown_stations.reset_index(drop=True, inplace=True)
unknown_stations.sort_index(inplace=True)

unknown_stations.sample(5)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,from_station_lat,from_station_lng,to_station_lat,to_station_lng
16692,120327,C4C4266011E78A6CE61235F5FE4F3EBA11666705,2016-11-25 18:29:55,2016-11-25 18:41:31,Kümmellstraße / Robert-Koch-Straße,140794,F PS 2423 / BSC Hamburg Mobil,238543,iPhone SRH,12,53.590237,9.986730,NaN,NaN
2070,117562,DAD455BAD6F07EFA0E0839A107D84BE0AF7E4CDF,2014-05-29 11:40:47,2014-05-29 12:14:48,Bahrenfelder Straße/Völckersstraße,211709,F PS 1763 / BSC Hamburg Mobil,225361,Terminal HH_98 (-2126-),35,53.556897,9.927945,NaN,NaN
6381,143755,5D72C6114931F67ECA26C2FC3601D94A8B896C7F,2015-02-19 19:23:20,2015-02-19 19:31:40,Uhlandstraße / Eingang Nord,138372,F R 5156 / BSC Hamburg Mobil,238540,Terminal HH_47 (-2315-),9,53.565000,10.026700,NaN,NaN
8251,120809,31F37E438103127954BA255E32E40E745BF6D22E,2015-04-18 14:45:05,2015-04-18 14:51:35,Hasselbrookstraße/Papenstraße,204031,F R 5156 / BSC Hamburg Mobil,238540,Terminal HH_17 (-2412-),7,53.564805,10.054679,NaN,NaN
14878,108102,C35027C23EB85EAC6B3B57FE3E2A270661EDBAF0,2015-12-08 13:16:27,2015-12-08 13:25:57,Harvestehuder Weg / Alte Rabenstraße,137994,F PS 2423 / BSC Hamburg Mobil,238543,Terminal HH_29 (-2213-),10,53.567200,10.001100,NaN,NaN


Die resultierenden Datensätze haben die folgende Anzahl ein Einträgen:

In [76]:
print("bookings: " + str(len(bookings.index)))
print("maintenance: " + str(len(maintenance.index)))
print("unknown_stations: " + str(len(unknown_stations.index)))

bookings: 8076273
maintenance: 728094
unknown_stations: 17471


Für die Visualisierung und weitere Analyse werden die Datensätze als `CSV` Dateien gespeichert.

In [77]:
bookings.to_csv("../data/call_a_bike/bookings.csv")
maintenance.to_csv("../data/call_a_bike/maintenance.csv")
unknown_stations.to_csv("../data/call_a_bike/unknown_stations.csv")